## Sistemas Lineares Singulares ou Não-quadrados

Considere a matriz abaixo.

In [3]:
A = [1.0 2.0 0.0 2.0 5.0; 0.0 0.0 1.0 -4.0 2.0]

2x5 Array{Float64,2}:
 1.0  2.0  0.0   2.0  5.0
 0.0  0.0  1.0  -4.0  2.0

$A$ claramente tem um problema. Ela só tem duas linhas, mas tem cinco colunas.
Então podemos esperar que $Ax = b$ não tenha solução, ou que tenha infinitas.
Mas como verificar isso?

In [4]:
A = [1.0 0.0; 0.0 1.0; 1.0 1.0]

3x2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0
 1.0  1.0

Esta $A$ tem um problema parecido. Suas linhas são em maior número que a coluna. O que esperar do sistema $Ax = b$?

## Imagem e Núcleo

Em Álgebra Linear, já foi visto imagem e núcleo de transformações lineares.
Também vimos que $A$ define a transformação $T:\mathbb{R}^n\longrightarrow\mathbb{R}^m$ dada
por $T(x) = Ax$. Para matrizes, estendendo o conceito, temos

$$ \mbox{Im}(A) = \{Ax \mid\ \forall x \in \mathbb{R}^n\}
= \{y \in \mathbb{R}^m \mid\ \exists x \in \mathbb{R}^m \mid\ y = Ax\}
\subset \mathbb{R}^m $$
$$ \mbox{Nu}(A) = \{x \mid\ Ax = 0\} \subset \mathbb{R}^n. $$

O sistema linear $Ax = b$ tem solução se $b \in \mbox{Im}(A)$. E caso tenha,
todas soluções são da forma $x + d$, onde $Ad = 0$, isto é, $x + \mbox{Nu}(A)$,
onde $x$ é uma solução.

No primeiro caso, podemos trocas as colunas de $A$ obtendo $[I\ B]$, onde $I$ é a identidade e $B$ é uma matriz qualquer. Trocamos as linhas de $x$ respectivamente, obtendo $[x_I; x_B]$. Uma solução do sistema $Ax = b$ seria
$x_I = b$ e $x_B = 0$.

De maneira geral, $I$ poderia ser $U$ e fariamos $x_I = U^{-1}b$.

Uma estratégia para obter isso é realmente fazer as permutações de coluna na fatoração LU.
Isso é chamado de pivoteamento completo.
Apesar de funcionar, é pouco praticado.

Def.: (Posto) O posto-linha da matriz $A$ é o número de linhas linearmente independente de $A$.
O posto-coluna é análogo.

Teo.: O posto-linha é igual ao posto-coluna e por isso dizemos apenas posto.

Def.: Dizemos que uma matriz $A: m\times n$ tem posto-linha completo de o posto de $A$ é $m$.
Análogo para posto-coluna completo.

Def.: Dizemos que $A$ tem posto completo se o posto de $A$ é $\min\{m,n\}$.

Teo.: Se $A$ tem posto-linha completo, então $Ax = b$ tem uma solução se $n = m$ ou infinitas
se $m < n$. (Note que $m > n$ é impossível.)

Teo.: Se $A$ tem posto-coluna completo, então $\mbox{Nu}(A) = \{0\}$. Daí, se existir solução,
ela é única. Note que pode não existir solução.

Teo.: Se $A$, $m\times n$ com $m \leq n$, então existem matrizes $L$, $U$ e $P$ tais que
$PA = LU$, com $P$ matriz de permutação de linhas, $L$ triangular inferior $m\times m$ com diagonal de 1s e
$U$ é tipo escada superior $m\times n$ com o mesmo posto que $A$.

In [1]:
A = [2.0 3.0 1.0 1.0 1.0; -4.0 -6.0 -3.0 3.0 2.0]

2x5 Array{Float64,2}:
  2.0   3.0   1.0  1.0  1.0
 -4.0  -6.0  -3.0  3.0  2.0

In [2]:
m21 = A[2,1]/A[1,1]
U = copy(A)
U[2,1] = 0.0
U[2,2:5] -= m21*U[1,2:5]
U

2x5 Array{Float64,2}:
 2.0  3.0   1.0  1.0  1.0
 0.0  0.0  -1.0  5.0  4.0

In [3]:
A = [2.0 3.0 1.0 1.0 1.0; 0.0 0.0 -1.0 5.0 4.0; 0.0 0.0 2.0 -10.0 -8.0]

3x5 Array{Float64,2}:
 2.0  3.0   1.0    1.0   1.0
 0.0  0.0  -1.0    5.0   4.0
 0.0  0.0   2.0  -10.0  -8.0

In [4]:
m33 = A[3,3]/A[2,3]
U = copy(A)
U[3,3] = 0.0
U[3,4:end] -= m33*U[2,4:end]
U

3x5 Array{Float64,2}:
 2.0  3.0   1.0  1.0  1.0
 0.0  0.0  -1.0  5.0  4.0
 0.0  0.0   0.0  0.0  0.0

Teo.: Se $A$ é $m \times n$ com $m > n$, então existe $L$, $m\times n$ com diagonal de 1
e $U$, $n \times n$ com mesmo posto que $A$, $P$ permutação de linhas tais que 
$PA = LU$.

In [5]:
A = rand(5,3)

5x3 Array{Float64,2}:
 0.611523    0.473446   0.0193772
 0.540549    0.042203   0.346138 
 0.00431418  0.0549514  0.461622 
 0.86256     0.195973   0.332966 
 0.0747737   0.907916   0.874113 

In [6]:
function dec_lu_pivot(A; tol=1e-12)
    m,n = size(A)
    U = copy(A)
    L = zeros(m,n)
    P = collect(1:m)
    for j = 1:n
        # Diferença/Correção
        if j > m
            break
        end
        i_pivo, pivo = j, abs(U[j,j])
        for i = j+1:m
            if abs(U[i,j]) > pivo
                pivo = abs(U[i,j])
                i_pivo = i
            end
        end
        if pivo < tol
            error("pivo ≈ 0.0")
        end
        if i_pivo != j
            U[[i_pivo; j],:] = U[[j; i_pivo],:]
            L[[i_pivo; j],:] = L[[j; i_pivo],:]
            P[[i_pivo; j]] = P[[j; i_pivo]]
        end
        L[j,j] = 1.0
        for i = j+1:m
            mij = U[i,j]/U[j,j]
            L[i,j] = mij
            U[i,j] = 0.0
            for k = j+1:n
                U[i,k] -= mij*U[j,k]
            end
        end
    end
    return L, U, P
end

dec_lu_pivot (generic function with 1 method)

In [8]:
# Testes
A = rand(3,5)
L, U, P = dec_lu_pivot(A)
#Está errado

(
3x5 Array{Float64,2}:
 1.0        0.0        0.0  0.0  0.0
 0.206779   1.0        0.0  0.0  0.0
 0.997316  -0.0920671  1.0  0.0  0.0,

3x5 Array{Float64,2}:
 0.975841  0.955795   0.450892   0.47079    0.55861  
 0.0       0.430349   0.861059   0.441267  -0.0780855
 0.0       0.0       -0.307227  -0.209151   0.271569 ,

[3,1,2])

In [10]:
A = rand(3,5)
A[3,:] = A[1,:] + A[2,:]
L, U, P = dec_lu_pivot(A)
# Posto linha incompleto -> pivo nulo

LoadError: LoadError: pivo ≈ 0.0
while loading In[10], in expression starting on line 3

In [11]:
A = rand(5,3)
L, U, P = dec_lu_pivot(A)

(
5x3 Array{Float64,2}:
 1.0       0.0        0.0     
 0.924039  1.0        0.0     
 0.735323  0.326429   1.0     
 0.734903  0.920646  -0.591073
 0.90758   0.473752  -0.965687,

5x3 Array{Float64,2}:
 0.929196  0.259956  0.571095
 0.0       0.618533  0.115076
 0.0       0.0       0.221869
 0.0       0.0       0.0     
 0.0       0.0       0.0     ,

[3,1,4,2,5])

In [12]:
A = rand(5,3)
A[:,3] = A[:,1] + A[:,2]
L, U, P = dec_lu_pivot(A)

LoadError: LoadError: pivo ≈ 0.0
while loading In[12], in expression starting on line 3

In [13]:
function dec_lu_pivot(A; tol=1e-12)
    m,n = size(A)
    U = copy(A)
    if m <= n
        L = zeros(m,m)
    else
        L = zeros(m,n)
    end
    P = collect(1:m)
    d = 1 # Linha onde escada deve ser não-nulo.
    # Se não tiver problemas, d=j, senão d<j.
    for j = 1:n
        i_pivo, pivo = d, abs(U[d,j])
        for i = d+1:m
            if abs(U[i,j]) > pivo
                pivo = abs(U[i,j])
                i_pivo = i
            end
        end
        if pivo < tol
            # Ninguém nessa coluna é não-nulo o suficiente.
            # Vamos dizer que são todos zero, então.
            for i = d:m
                U[i,j] = 0.0
            end
            continue
        end
        if i_pivo != d
            U[[i_pivo; d],:] = U[[d; i_pivo],:]
            L[[i_pivo; d],:] = L[[d; i_pivo],:]
            P[[i_pivo; d]] = P[[d; i_pivo]]
        end
        for i = d+1:m
            mid = U[i,j]/U[d,j]
            L[i,d] = mid
            U[i,j] = 0.0
            for k = j+1:n
                U[i,k] -= mid*U[d,k]
            end
        end
        d += 1
        if d > m
            break
        end
    end
    for i = 1:min(m,n)
        L[i,i] = 1.0
    end
    if m > n
        U = U[1:n,:]
    end
    return L, U, P
end

dec_lu_pivot (generic function with 1 method)

In [15]:
# Testes
A = rand(3,5)
L, U, P = dec_lu_pivot(A)

(
3x3 Array{Float64,2}:
 1.0        0.0       0.0
 0.955345   1.0       0.0
 0.427074  -0.827266  1.0,

3x5 Array{Float64,2}:
 0.917032   0.747054   0.845446   0.813639   0.756309 
 0.0       -0.607503  -0.401594  -0.594078  -0.0921091
 0.0        0.0       -0.481742  -0.397093   0.185504 ,

[1,3,2])

In [16]:
L*U - A[P,:]

3x5 Array{Float64,2}:
 0.0  0.0  0.0   0.0          0.0
 0.0  0.0  0.0   0.0          0.0
 0.0  0.0  0.0  -5.55112e-17  0.0

In [17]:
A = rand(3,5)
A[3,:] = A[1,:] + A[2,:]
L, U, P = dec_lu_pivot(A)

(
3x3 Array{Float64,2}:
 1.0        0.0  0.0
 0.307865   1.0  0.0
 0.692135  -1.0  1.0,

3x5 Array{Float64,2}:
 1.24186  0.944316  1.31056    0.826033  1.82702 
 0.0      0.289339  0.436767  -0.107755  0.300349
 0.0      0.0       0.0        0.0       0.0     ,

[3,2,1])

In [18]:
L*U - A[P,:]

3x5 Array{Float64,2}:
 0.0  0.0   0.0          0.0  0.0
 0.0  0.0   0.0          0.0  0.0
 0.0  0.0  -5.55112e-17  0.0  0.0

In [25]:
A = rand(5,7)
A[3,1:6] = 0.0
A[4,1:4] = 0.0
A[5,1:3] = 0.0
A[3,:] += A[1,:]
A[4,:] += A[1,:] + 2*A[2,:]
A[5,:] += -A[1,:] + 3*A[2,:]
A

5x7 Array{Float64,2}:
 0.145305  0.298467  0.823932  0.553674  0.0115993  0.666618  0.173672
 0.783909  0.706531  0.880038  0.67816   0.525609   0.899663  0.263982
 0.145305  0.298467  0.823932  0.553674  0.0115993  0.666618  0.690851
 1.71312   1.71153   2.58401   1.90999   1.6937     3.05059   1.02258 
 2.20642   1.82113   1.81618   2.45816   1.74591    2.4751    1.16144 

In [26]:
L, U, P = dec_lu_pivot(A)

(
5x5 Array{Float64,2}:
 1.0        0.0   0.0   0.0  0.0
 0.776426   1.0   0.0   0.0  0.0
 0.0658554  0.6   1.0   0.0  0.0
 0.355285   0.2  -0.5   1.0  0.0
 0.0658554  0.6   1.0  -0.0  1.0,

5x7 Array{Float64,2}:
 2.20642  1.82113   1.81618  2.45816      1.74591    2.4751     1.16144  
 0.0      0.297559  1.17388  0.00141456   0.338131   1.12886    0.120809 
 0.0      0.0       0.0      0.390942    -0.306257  -0.173695   0.541878 
 0.0      0.0       0.0      0.0         -0.31544   -0.292322   0.0981158
 0.0      0.0       0.0      0.0          0.0        0.0       -0.517178 ,

[5,4,3,2,1])

In [27]:
L*U - A[P,:]

5x7 Array{Float64,2}:
 0.0          0.0   0.0           0.0          0.0  0.0           0.0        
 0.0          0.0   0.0           0.0          0.0  0.0           0.0        
 0.0          0.0  -3.33067e-16   0.0          0.0  0.0           0.0        
 1.11022e-16  0.0   1.11022e-16  -1.11022e-16  0.0  2.22045e-16  -5.55112e-17
 0.0          0.0  -3.33067e-16   0.0          0.0  0.0           0.0        

In [30]:
A = rand(5,7)
A[1,1:6] = 0.0
A[2,1:3] = 0.0
A[2,5:end] = 0.0
A[3,1:5] = 0.0
A[1,:] += A[4,:]
A[2,:] += A[5,:] + 2*A[4,:]
A[3,:] += -A[5,:] + 3*A[4,:]
A

5x7 Array{Float64,2}:
 0.531134  0.466038  0.236221  0.687358  0.94164   0.329443  1.09771  
 1.17776   1.80798   0.892776  2.44863   2.30706   1.24873   0.476631 
 1.47791   0.522211  0.288331  1.34583   2.40114   1.30246   1.42921  
 0.531134  0.466038  0.236221  0.687358  0.94164   0.329443  0.188952 
 0.115487  0.875903  0.420333  0.716245  0.423776  0.589845  0.0987267

In [32]:
L, U, P = dec_lu_pivot(A)

(
5x5 Array{Float64,2}:
 1.0        0.0  0.0       0.0  0.0
 0.796904   1.0  0.0       0.0  0.0
 0.0781422  0.6  1.0       0.0  0.0
 0.359381   0.2  0.333333  1.0  0.0
 0.359381   0.2  0.333333  1.0  1.0,

5x7 Array{Float64,2}:
 1.47791  0.522211  0.288331   1.34583   2.40114       1.30246    1.42921 
 0.0      1.39183   0.663004   1.37613   0.393576      0.210795  -0.662309
 0.0      0.0       0.0       -0.2146   -8.32667e-17   0.361591   0.384431
 0.0      0.0       0.0        0.0       0.0          -0.301326  -0.320359
 0.0      0.0       0.0        0.0       0.0           0.0        0.908761,

[3,2,5,4,1])

In [33]:
L*U - A[P,:]

5x7 Array{Float64,2}:
 0.0   0.0          0.0  0.0  0.0  0.0          0.0
 0.0   0.0          0.0  0.0  0.0  0.0          0.0
 0.0  -1.11022e-16  0.0  0.0  0.0  0.0          0.0
 0.0  -5.55112e-17  0.0  0.0  0.0  5.55112e-17  0.0
 0.0  -5.55112e-17  0.0  0.0  0.0  5.55112e-17  0.0

In [35]:
A = rand(5,3)
L, U, P = dec_lu_pivot(A)

(
5x3 Array{Float64,2}:
 1.0        0.0        0.0     
 0.597085   1.0        0.0     
 0.164092  -0.0396259  1.0     
 0.024956   0.432991   0.343885
 0.596066  -0.282519   0.413941,

3x3 Array{Float64,2}:
 0.36945  0.470272  0.273646
 0.0      0.659986  0.38457 
 0.0      0.0       0.938008,

[1,5,4,3,2])

In [36]:
L*U - A[P,:]

5x3 Array{Float64,2}:
  0.0           0.0          0.0        
  0.0           0.0          0.0        
 -6.93889e-18   0.0          0.0        
  0.0           0.0          5.55112e-17
  2.77556e-17  -1.38778e-17  0.0        

In [38]:
A = rand(5,3)
A[:,3] = A[:,1] + A[:,2]
A

5x3 Array{Float64,2}:
 0.594645   0.366137  0.960781
 0.505058   0.838961  1.34402 
 0.0119654  0.970835  0.9828  
 0.495741   0.478554  0.974295
 0.48984    0.86826   1.3581  

In [39]:
L, U, P = dec_lu_pivot(A)

(
5x3 Array{Float64,2}:
 1.0        0.0       0.0
 0.0201219  1.0       0.0
 0.849344   0.548004  1.0
 0.833676   0.179887  0.0
 0.823753   0.58814   0.0,

3x3 Array{Float64,2}:
 0.594645  0.366137  0.960781
 0.0       0.963468  0.963468
 0.0       0.0       0.0     ,

[1,3,2,4,5])

In [40]:
L*U - A[P,:]

5x3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [41]:
A = rand(9,5)
A[1,:] = 0
A[:,1] = 0
A[:,3] = -2*A[:,4]
A[5,:] = A[4,:] + A[6,:]
A[7,3:5] = 0
A[7,:] += A[8,:]
A

9x5 Array{Float64,2}:
 0.0  0.0        -0.0        0.0        0.0     
 0.0  0.243955   -1.26429    0.632146   0.87926 
 0.0  0.04042    -0.865571   0.432785   0.914434
 0.0  0.974076   -0.0314446  0.0157223  0.592016
 0.0  1.06102    -0.0768298  0.0384149  1.10412 
 0.0  0.0869388  -0.0453852  0.0226926  0.512102
 0.0  1.33228    -1.7404     0.870199   0.33262 
 0.0  0.750551   -1.7404     0.870199   0.33262 
 0.0  0.898932   -0.40829    0.204145   0.19969 

In [43]:
L, U, P = dec_lu_pivot(A)

(
9x5 Array{Float64,2}:
 1.0         0.0         0.0       0.0  0.0
 0.796393    1.0         0.0       0.0  0.0
 0.0303391  -0.620808    1.0       0.0  0.0
 0.731137    0.947918   -0.313391  1.0  0.0
 0.183112   -0.722271    0.999411  0.0  1.0
 0.0652559   0.0520817   0.313391  0.0  0.0
 0.0         0.0         0.0       0.0  0.0
 0.56336    -0.580446    0.443655  0.0  0.0
 0.674734    0.585097   -0.361855  0.0  0.0,

5x5 Array{Float64,2}:
 0.0  1.33228  -1.7404    0.870199  0.33262
 0.0  0.0       1.30921  -0.654606  0.83922
 0.0  0.0       0.0       0.0       1.42534
 0.0  0.0       0.0       0.0       0.0    
 0.0  0.0       0.0       0.0       0.0    ,

[7,5,3,4,2,6,1,8,9])

In [44]:
L*U - A[P,:]

9x5 Array{Float64,2}:
 0.0  0.0          0.0           0.0           0.0        
 0.0  0.0          0.0           0.0           0.0        
 0.0  0.0          0.0           0.0          -2.22045e-16
 0.0  1.11022e-16  2.77556e-17  -1.38778e-17   1.11022e-16
 0.0  0.0          0.0           0.0           1.11022e-16
 0.0  0.0          6.93889e-18  -3.46945e-18   0.0        
 0.0  0.0          0.0           0.0           0.0        
 0.0  0.0          0.0           0.0           0.0        
 0.0  0.0          0.0           0.0           0.0        

É possível reduzir ainda mais.

In [46]:
L = tril(rand(8,3),-1) + eye(8,3)

8x3 Array{Float64,2}:
 1.0        0.0       0.0     
 0.959195   1.0       0.0     
 0.323802   0.26421   1.0     
 0.480774   0.95311   0.784777
 0.0419428  0.31787   0.528203
 0.73864    0.667382  0.530871
 0.888458   0.202518  0.325625
 0.718781   0.875378  0.856977

In [47]:
U = triu(rand(3,8))

3x8 Array{Float64,2}:
 0.809961  0.0979328  0.963857   0.960599  …  0.299834  0.763872  0.234869 
 0.0       0.835831   0.0578628  0.348888     0.266856  0.245129  0.0619789
 0.0       0.0        0.862253   0.697509     0.549231  0.530045  0.935909 

In [48]:
A = L*U

8x8 Array{Float64,2}:
 0.809961   0.0979328  0.963857  0.960599  …  0.299834  0.763872  0.234869
 0.776911   0.929767   0.982389  1.27029      0.554455  0.977831  0.287264
 0.262267   0.252546   1.18964   1.10073      0.716824  0.842154  1.02834 
 0.389408   0.843722   1.19522   1.34175      0.829519  1.01685   0.906471
 0.0339721  0.269793   0.514265  0.519617     0.387507  0.389929  0.523903
 0.59827    0.630155   1.20831   1.31267   …  0.691135  1.00921   0.711695
 0.719616   0.25628    1.14884   1.15123      0.499276  0.900907  0.525978
 0.582185   0.80206    1.48239   1.59362      0.919794  1.21787   1.02513 

In [49]:
L, U, P = dec_lu_pivot(A)

(
8x8 Array{Float64,2}:
 1.0        0.0       0.0       0.0  0.0  0.0  0.0  0.0
 0.959195   1.0       0.0       0.0  0.0  0.0  0.0  0.0
 0.323802   0.26421   1.0       0.0  0.0  0.0  0.0  0.0
 0.480774   0.95311   0.784777  1.0  0.0  0.0  0.0  0.0
 0.0419428  0.31787   0.528203  0.0  1.0  0.0  0.0  0.0
 0.73864    0.667382  0.530871  0.0  0.0  1.0  0.0  0.0
 0.888458   0.202518  0.325625  0.0  0.0  0.0  1.0  0.0
 0.718781   0.875378  0.856977  0.0  0.0  0.0  0.0  1.0,

8x8 Array{Float64,2}:
 0.809961  0.0979328  0.963857   0.960599  …  0.299834  0.763872  0.234869 
 0.0       0.835831   0.0578628  0.348888     0.266856  0.245129  0.0619789
 0.0       0.0        0.862253   0.697509     0.549231  0.530045  0.935909 
 0.0       0.0        0.0        0.0          0.0       0.0       0.0      
 0.0       0.0        0.0        0.0          0.0       0.0       0.0      
 0.0       0.0        0.0        0.0       …  0.0       0.0       0.0      
 0.0       0.0        0.0        0.0          0.

In [50]:
A = rand(3,5)
L, U, P = dec_lu_pivot(A)

(
3x3 Array{Float64,2}:
 1.0       0.0      0.0
 0.71543   1.0      0.0
 0.588322  0.44017  1.0,

3x5 Array{Float64,2}:
 0.680453  0.255885   0.692408   0.0442224   0.214487 
 0.0       0.471634  -0.0701619  0.327921   -0.0108661
 0.0       0.0        0.394022   0.336101    0.0943811,

[1,3,2])

In [51]:
Util = U[:,1:3]
B = U[:,4:end]

3x2 Array{Float64,2}:
 0.0442224   0.214487 
 0.327921   -0.0108661
 0.336101    0.0943811

In [53]:
# Resolve os sistemas
V = -Util\B

3x2 Array{Float64,2}:
  1.11218   -0.0667353
 -0.822184  -0.0125945
 -0.853     -0.239533 

In [56]:
a, b = 2,1

(2,1)

In [59]:
b = V*[a;b]

LoadError: LoadError: DimensionMismatch("second dimension of A, 2, does not match length of x, 4")
while loading In[59], in expression starting on line 1

In [58]:
A\b

5-element Array{Float64,1}:
  3.47094 
 -3.31634 
  1.1452  
 -5.6101  
  0.464179